In [1]:
using IntervalArithmetic,IntervalBoxes,IntervalArithmetic.Symbols,Plots

In [2]:
x1 = 1 / 5
x2 = 2 / 5
x3 = 3 / 5
x4 = 4 / 5
h = 1
g = 9.8
function calculate_expression(y1::Interval{T}, y2::Interval{T}, y3::Interval{T}, y4::Interval{T}) where T
    term1 = sqrt(x1^2 + (y1 - h)^2) / sqrt(2 * g * (h - y1))
    term2 = sqrt((x1 - x2)^2 + (y1 - y2)^2) / (sqrt(2 * g * (h - y1)) + sqrt(2 * g * (h - y2)))
    term3 = sqrt((y2 - y3)^2 + (x2 - x3)^2) / (sqrt(2 * g * (h - y2)) + sqrt(2 * g * (h - y3)))
    term4 = sqrt((y3 - y4)^2 + (x3 - x4)^2) / (sqrt(2 * g * (h - y3)) + sqrt(2 * g * (h - y4)))
    term5 =  sqrt(y4^2 + (x4 - h)^2) / (sqrt(2 * g * (h - y4)) + sqrt(2 * g * h))
    return 2*(term1 + term2 + term3 + term4 + term5)
end

calculate_expression (generic function with 1 method)

In [ ]:
function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, Y3::Interval{T}, Y4::Interval{T}) where T
    interval_lists = [[Y1, Y2, Y3, Y4]]
    upper_bound = sup(f(Y1..., Y2..., Y3..., Y4...))
    upper_bounds = [upper_bound]
    working_list = [(Y1..., Y2..., Y3..., Y4...)]
    while !isempty(working_list)
        (current_Y1, current_Y2, current_Y3, current_Y4) = popfirst!(working_list)
        if diam(current_Y1) <= 0.005 && diam(current_Y2) <= 0.005 && diam(current_Y3) <= 0.001 && diam(current_Y4) <= 0.005
            push!(working_list, (current_Y1, current_Y2, current_Y3, current_Y4))
            #push!(solution, (current_Y1, current_Y2, current_Y3, current_Y4))
            break
        end
        upper_bound = min(upper_bound, sup(f(interval(mid(current_Y1))..., interval(mid(current_Y2))..., interval(mid(current_Y3))..., interval(mid(current_Y4))...)))
        if inf(f(current_Y1..., current_Y2..., current_Y3..., current_Y4)) <= upper_bound
            if(diam(current_Y1)>= diam(current_Y2) && diam(current_Y1)>= diam(current_Y3) && diam(current_Y1)>= diam(current_Y4)) && diam(current_Y1) >= 0.005
                S = bisect(current_Y1)
                push!(working_list, (S[1], current_Y2, current_Y3, current_Y4), (S[2], current_Y2, current_Y3, current_Y4))
            elseif (diam(current_Y2)>= diam(current_Y1)) && diam(current_Y2)>= diam(current_Y3) && diam(current_Y2) >= diam(current_Y4) && diam(current_Y2) >= 0.005
                S1 = bisect(current_Y2)
                push!(working_list, (current_Y1, S1[1], current_Y3, current_Y4), (current_Y1, S1[2], current_Y3, current_Y4))
            elseif diam(current_Y3)>= diam(current_Y4) && diam(current_Y3) >= 0.005
                S2 = bisect(current_Y3)
                push!(working_list, (current_Y1, current_Y2, S2[1], current_Y4), (current_Y1, current_Y2, S2[2], current_Y4))
            elseif diam(current_Y4) >= 0.005
                S3 = bisect(current_Y4)
                push!(working_list, (current_Y1, current_Y2,current_Y3, S3[1]), (current_Y1, current_Y2,current_Y3, S3[2]))
            else
                push!(working_list, (current_Y1, current_Y2, current_Y3, current_Y4))
            end
        end
        push!(upper_bounds, upper_bound)
    end

    return working_list, upper_bounds, solution
end

calculate_branch_bound (generic function with 2 methods)

In [3]:
function calculate_branch_bound(f, Y1::Interval{T}, Y2::Interval{T}, Y3::Interval{T}, Y4::Interval{T}, N) where T
    interval_lists = [[Y1, Y2, Y3, Y4]]
    upper_bound = sup(f(Y1..., Y2..., Y3..., Y4...))
    upper_bounds = [upper_bound]
    working_list = [(Y1..., Y2..., Y3..., Y4...)]
    for i in 1:N
        if isempty(working_list)
            break
        end
        (current_Y1, current_Y2, current_Y3, current_Y4) = popfirst!(working_list)

        #upper_bound = min(upper_bound, sup(f(current_Y1..., current_Y2..., current_Y3..., current_Y4...)))
        upper_bound = min(upper_bound, sup(f(interval(mid(current_Y1))..., interval(mid(current_Y2))..., interval(mid(current_Y3))..., interval(mid(current_Y4))...)))
        if inf(f(current_Y1..., current_Y2..., current_Y3..., current_Y4)) <= upper_bound
            if(diam(current_Y1)>= diam(current_Y2) && diam(current_Y1)>= diam(current_Y3) && diam(current_Y1)>= diam(current_Y4)) && diam(current_Y1) >= 0.001
                S = bisect(current_Y1)
                push!(working_list, (S[1], current_Y2, current_Y3, current_Y4), (S[2], current_Y2, current_Y3, current_Y4))
            elseif (diam(current_Y2)>= diam(current_Y1)) && diam(current_Y2)>= diam(current_Y3) && diam(current_Y2) >= diam(current_Y4) && diam(current_Y2) >= 0.001
                S1 = bisect(current_Y2)
                push!(working_list, (current_Y1, S1[1], current_Y3, current_Y4), (current_Y1, S1[2], current_Y3, current_Y4))
            elseif diam(current_Y3)>= diam(current_Y4) && diam(current_Y3) >= 0.001
                S2 = bisect(current_Y3)
                push!(working_list, (current_Y1, current_Y2, S2[1], current_Y4), (current_Y1, current_Y2, S2[2], current_Y4))
            elseif diam(current_Y4) >= 0.001
                S3 = bisect(current_Y4)
                push!(working_list, (current_Y1, current_Y2,current_Y3, S3[1]), (current_Y1, current_Y2,current_Y3, S3[2]))
            else
                push!(working_list, (current_Y1, current_Y2, current_Y3, current_Y4))
            end
        end

        push!(upper_bounds, upper_bound)
    end
    return working_list, upper_bounds
end

calculate_branch_bound (generic function with 1 method)

In [4]:
y1_interval = interval(0.0, 1)
y2_interval = interval(0.0, 1)
y3_interval = interval(0.0, 1)
y4_interval = interval(0.0, 1)
N = 1000000
working_list, minimas = calculate_branch_bound(calculate_expression, y1_interval, y2_interval,y3_interval, y4_interval, N)
#working_list, minimas = calculate_branch_bound(calculate_expression, y1_interval, y2_interval, y3_interval, y4_interval)


(NTuple{4, Interval{Float64}}[([0.578124, 0.585938]_com, [0.367187, 0.375001]_com, [0.218749, 0.226563]_com, [0.0703124, 0.0781251]_com), ([0.578124, 0.585938]_com, [0.367187, 0.375001]_com, [0.226562, 0.234376]_com, [0.0625, 0.0703126]_com), ([0.578124, 0.585938]_com, [0.367187, 0.375001]_com, [0.226562, 0.234376]_com, [0.0703124, 0.0781251]_com), ([0.585937, 0.593751]_com, [0.359374, 0.367188]_com, [0.218749, 0.226563]_com, [0.0625, 0.0703126]_com), ([0.585937, 0.593751]_com, [0.359374, 0.367188]_com, [0.218749, 0.226563]_com, [0.0703124, 0.0781251]_com), ([0.585937, 0.593751]_com, [0.359374, 0.367188]_com, [0.226562, 0.234376]_com, [0.0625, 0.0703126]_com), ([0.585937, 0.593751]_com, [0.359374, 0.367188]_com, [0.226562, 0.234376]_com, [0.0703124, 0.0781251]_com), ([0.585937, 0.593751]_com, [0.367187, 0.375001]_com, [0.218749, 0.226563]_com, [0.0625, 0.0703126]_com), ([0.585937, 0.593751]_com, [0.367187, 0.375001]_com, [0.218749, 0.226563]_com, [0.0703124, 0.0781251]_com), ([0.585937

In [46]:
begin
    z = calculate_expression(interval(mid(0.328124, 0.343751)),interval(mid(0.187499, 0.203126)),interval(mid(0.0781249, 0.0937501)),interval(mid(0.03125, 0.0625)))
    z1 = calculate_expression(interval(0.656249, 0.687501),interval(0.437499, 0.468751),interval(0.406249, 0.437501),interval(0.0, 0.0625))
    z2 = calculate_expression(interval(0.578124, 0.582032),interval(0.359374, 0.367188),interval(0.218749, 0.226563),interval(0.0703124, 0.0781251))
    println(z)
    println(z1)
    println(z2)
end

[0.599658, 0.599659]_com_NG
[0.58462, 0.67227]_com_NG
[0.585756, 0.598993]_com_NG
